# Athena audits: demo and state results
Work out test cases for multi-round audits.

Show usage of the Audit class.

Demo 2016 presidential contest in selected states

## Setup and define some utilities

In [1]:
from athena.audit import Audit
import math
import json
import sys

In [2]:
# Run this any time for fast turnaround of changes to the library 
from importlib import reload
import athena.audit
reload(athena.audit)
from athena.audit import Audit

In [3]:
def make_audit(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule):
    "Convenience function to make Audit instance with given election parameters"

    election = {
        "alpha": alpha,
        "delta": delta,
        "candidates": candidates,
        "results": results,
        "ballots_cast": ballots_cast,
        "winners": winners,
        "name": name,
        "model": model,
        "pstop_goal": pstop_goal,
    }
    a = Audit(audit_type, election['alpha'], election['delta'])
    a.add_election(election)
    a.add_round_schedule(round_schedule)
    return a

In [4]:
def find_next_round_size(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule):
    "Convenience function to call a fresh Audit instance with given election parameters"

    election = {
        "alpha": alpha,
        "delta": delta,
        "candidates": candidates,
        "results": results,
        "ballots_cast": ballots_cast,
        "winners": winners,
        "name": name,
        "model": model,
        "pstop_goal": pstop_goal,
    }
    a = Audit(audit_type, election['alpha'], election['delta'])
    a.add_election(election)
    a.add_round_schedule(round_schedule)
    x = a.find_next_round_size(election['pstop_goal'])
    return x

In [5]:
def sample90(margin, audit_type="ATHENA"):
    "Return sample size and other output given margin"

    assert 0.0 < margin < 1.0
    ballots_cast = 10000
    margin_votes = round(margin * ballots_cast)
    b = ballots_cast//2 - margin_votes // 2
    a = b + int(margin_votes)
    x = find_next_round_size(audit_type, 0.1, 1.0, ["A", "B"], [a, b], ballots_cast, 1, "state", "bin", [0.9], [])
    return (x['detailed']['A-B']['next_round_sizes'][0], x)

In [6]:
def sample90v(a, b, ballots_cast, audit_type="ATHENA"):
    "Return sample size etc. given votes for each of top two candidates in 1-winner contest"

    winner = max(a,b)
    loser = min(a,b)
    assert 0 < loser < winner < ballots_cast
    x = find_next_round_size(audit_type, 0.1, 1.0, ["A", "B"], [winner, loser], ballots_cast, 1, "state", "bin", [0.9], [])
    return (x['detailed']['A-B']['next_round_sizes'][0], x)

In [7]:
# Define a class to avoid cluttering notebook with stdout
class redirect_output(object):
    """context manager for reditrecting stdout/err to files"""


    def __init__(self, stdout='', stderr=''):
        self.stdout = stdout
        self.stderr = stderr

    def __enter__(self):
        self.sys_stdout = sys.stdout
        self.sys_stderr = sys.stderr

        if self.stdout:
            sys.stdout = open(self.stdout, 'w')
        if self.stderr:
            if self.stderr == self.stdout:
                sys.stderr = sys.stdout
            else:
                sys.stderr = open(self.stderr, 'w')

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout = self.sys_stdout
        sys.stderr = self.sys_stderr

In [8]:
# Print all the current properties and values of an object
# From https://stackoverflow.com/a/59128615/507544
from pprint import pprint
from inspect import getmembers
from types import FunctionType

def attributes(obj):
    disallowed_names = {
      name for name, value in getmembers(type(obj))
        if isinstance(value, FunctionType)}
    return {
      name: getattr(obj, name) for name in dir(obj) 
        if name[0] != '_' and name not in disallowed_names and hasattr(obj, name)}

def print_attributes(obj):
    "print all the current properties and values of an object"

    pprint(attributes(obj))

# Basic demo of multi-round Audit
Simple recipe for test with multiple rounds: exactly half of selected ballots are for declared winner, naively leading to an ever-escalating audit

In [9]:
audit_type = "ATHENA"
alpha = 0.1
delta = 1.0
candidates = ["A", "B"]
results = [60000, 40000]
ballots_cast = 100000
winners = 1
name = "test_election"
model = "bin"
pstop_goal = [.7, .9]
round_schedule = []

In [10]:
a = make_audit(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule)

## Round 1: select 112 ballots (70% stopping probability), 56 of which are for winner

In [11]:
x = a.find_next_round_size(pstop_goal)

In [12]:
x

{'detailed': {'A-B': {'pstop_goal': [0.7, 0.9],
   'next_round_sizes': [112, 184],
   'prob_stop': [0.70027553974696566, 0.90920677012971252]}},
 'future_round_sizes': [112, 184]}

Take the first offered sample size, for 70% stopping probability

In [13]:
sample_size = x['future_round_sizes'][0]

In [14]:
sample_size

112

In [15]:
a.add_round_schedule([sample_size])

In [16]:
winner_shares = [sample_size // 2]

In [17]:
r = a.find_risk(winner_shares)

In [18]:
r

{'risk': 0.54448293810382209,
 'delta': 9.8358285480178225,
 'passed': 0,
 'observed': [56],
 'required': [65]}

In [19]:
def next_round(a, winner_shares, r):
    below_kmin = max(r['required']) - max(r['observed'])
    x = a.find_next_round_size(pstop_goal)
    incremental_round_sizes = list(map(lambda x: x - max(a.round_schedule) + 2 * below_kmin, x['future_round_sizes']))
    incremental_sample_size = incremental_round_sizes[0]
    a.add_round_schedule(a.round_schedule + [max(a.round_schedule) + incremental_sample_size])
    next_total_winner_share = a.round_schedule[-1] // 2
    winner_shares += [next_total_winner_share]
    print(f'Next round: select {incremental_sample_size} more ballots, next total winner share is {next_total_winner_share}')
    r = a.find_risk(winner_shares)
    return r

## Round 2: select 132 more ballots, half of which are for winner

In [20]:
r = next_round(a, winner_shares, r)

Next round: select 132 more ballots, next total winner share is 122


In [21]:
a.round_schedule

[112, 244]

In [22]:
r

{'risk': 0.54448293810382209,
 'delta': 145.5156049638249,
 'passed': 0,
 'observed': [56, 122],
 'required': [65, 137]}

## Round 3: select 174 more ballots, half for winner

In [23]:
r = next_round(a, winner_shares, r)

Next round: select 173 more ballots, next total winner share is 208


In [24]:
a.round_schedule

[112, 244, 417]

In [25]:
r

{'risk': 0.54448293810382209,
 'delta': 6088.2649067227831,
 'passed': 0,
 'observed': [56, 122, 208],
 'required': [65, 137, 231]}

## Re-imagine last final winner share
`passed` should be true with the required winner share, false with one less

In [26]:
a.find_risk(winner_shares[:-1] + [r['required'][-1]])

{'risk': 0.094413126280234627,
 'delta': 0.54249355438260582,
 'passed': 1,
 'observed': [56, 122, 231],
 'required': [65, 137, 231]}

In [27]:
a.find_risk(winner_shares[:-1] + [r['required'][-1] - 1])

{'risk': 0.12556111897870845,
 'delta': 0.81374033157390635,
 'passed': 0,
 'observed': [56, 122, 230],
 'required': [65, 137, 231]}

In [28]:
a.election.results

[60000, 40000]

# Repeated tests

# Mercer County Audit

Mercer County Courthouse, Monday, 2019-11-18

https://www.mcc.co.mercer.pa.us/election/Election.Results/2019/GENERAL/SUMMARY.pdf

```
random seed: 62517875988999836270
Arlo p-value: 0.00058828
Yes: 15,042
No: 5,275
Ballots: 23,667
```

PROPOSED CONSTITUTIONAL AMENDMENT/CRIME VICTIM RIGHTS
aka Marsy's law

contests = {'victims': {'Yes': 15038, 'No': 5274, 'ballots': 23662, 'numWinners': 1}}

3350 blank = 23662-15038-5274

out of 80 total sampled, winner: 49 loser: 18 invalid: 13

News articles

RIRLA docs for background



In [92]:
mercer_ballots = 23662

In [86]:
mercer_yes = 15038
mercer_no = 5274

In [101]:
mercer_undervote = mercer_ballots - mercer_yes - mercer_no

In [102]:
mercer_undervote

3350

In [99]:
mercer_yes / mercer_ballots

0.6355337672217057

In [100]:
mercer_no / mercer_ballots

0.22288902037021385

In [103]:
mercer_undervote / mercer_ballots

0.14157721240808047

In [104]:
# Sample results
m_sample = 80
m_w = 49
m_l = 18
m_u = 13

In [106]:
[f'{v/m_sample:.2%}' for v in [49, 18, 13]]

['61.25%', '22.50%', '16.25%']

Winner share of pairwise vote

In [98]:
mercer_yes / (mercer_yes + mercer_no)

0.7403505317053958

In [90]:
mercer_pairwise_margin = (mercer_yes - mercer_no) / (mercer_yes + mercer_no)

In [91]:
mercer_pairwise_margin

0.48070106341079166

$ rlacalc -p -m 48.070106341079166

Sample size = 21 for ballot polling, margin 48.0701%, risk 10%

In [95]:
mercer_dilution = (mercer_yes + mercer_no) / mercer_ballots

In [96]:
mercer_dilution

0.8584227875919195

Sample size 25 for BRAVO ASN, expanded to account for undervotes:

In [97]:
21 / mercer_dilution

24.46346987002757

## Athena results

In [122]:
audit_type = "ATHENA"
alpha = 0.1
delta = 1.0
candidates = ["Yes", "No"]
results = [mercer_yes, mercer_no]
ballots_cast = mercer_ballots
winners = 1
name = "test_election"
model = "bin"
pstop_goal = [.7, .8, .9]
round_schedule = []

In [123]:
a = make_audit(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule)

In [124]:
x = a.find_next_round_size(pstop_goal)

In [125]:
x

{'detailed': {'Yes-No': {'pstop_goal': [0.7, 0.8, 0.9],
   'next_round_sizes': [24, 26, 39],
   'prob_stop': [0.75447131357743447,
    0.81044106054946019,
    0.93654357756080742]}},
 'future_round_sizes': [24, 26, 39]}

In [128]:
sample_size = 80

In [130]:
winner_shares = [49]
loser_shares = [18]

In [131]:
relevant_sample_size = winner_shares[0] + loser_shares[0]

In [132]:
relevant_sample_size

67

In [135]:
a.add_round_schedule([relevant_sample_size])

In [136]:
a.round_schedule


[67]

In [137]:
r = a.find_risk(winner_shares)

In [138]:
r

{'risk': 0.00015461627550637905,
 'delta': 0.0005882801239972858,
 'passed': 1,
 'observed': [49],
 'required': [36]}

Here is the diluted margin, which isn't relevant for BRAVO pairwise auditing

In [93]:
mercer_diluted_margin = (mercer_yes - mercer_no) / (mercer_ballots)

In [94]:
mercer_diluted_margin

0.41264474685149183

## Arlo estimates

In [140]:
# Not sure what is going wrong here. The find_next_round_size just loops....

In [120]:
a = make_audit("arlo", alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule)

In [121]:
a.find_next_round_size(pstop_goal)

KeyboardInterrupt: 

# Demo with 100 irrelevant ballots
Show a bit of performance / timing info also

FIXME: Needs more checking and updating, I think....

In [29]:
audit_type = "ATHENA"
alpha = 0.1
delta = 1.0
candidates = ["A", "B"]
results = [600, 300]
ballots_cast = 1000
winners = 1
name = "test_election"
model = "bin"
pstop_goal = [.5, .7, .9]
round_schedule = []

In [30]:
a = make_audit(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule)

In [31]:
%time x = a.find_next_round_size(pstop_goal)

CPU times: user 46.7 ms, sys: 0 ns, total: 46.7 ms
Wall time: 47 ms


In [32]:
x

{'detailed': {'A-B': {'pstop_goal': [0.5, 0.7, 0.9],
   'next_round_sizes': [32, 48, 69],
   'prob_stop': [0.53551331360561272,
    0.76101184637116392,
    0.90211353611617051]}},
 'future_round_sizes': [32, 48, 69]}

In [33]:
%%timeit
x = a.find_next_round_size(pstop_goal)

23.1 ms ± 2.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
sample_size = x['future_round_sizes'][0]

In [35]:
a.add_round_schedule([sample_size])

In [36]:
a.round_schedule

[32]

In [37]:
r = a.find_risk([sample_size // 2])

In [38]:
r

{'risk': 0.5794753786910346,
 'delta': 6.5832501720274239,
 'passed': 0,
 'observed': [16],
 'required': [19]}

In [39]:
sample_size

32

In [40]:
[(w, a.find_risk([w])['risk']) for w in range(0, sample_size, 10)]

[(0, 0.99999999999999822),
 (10, 0.98997857237158082),
 (20, 0.14215780336799913),
 (30, 0.00037687470333555909)]

In [41]:
winner_share_big_win = 2 * sample_size // 3

In [42]:
r = a.find_risk([winner_share_big_win])

In [43]:
r

{'risk': 0.087505775396554228,
 'delta': 0.20572656787585672,
 'passed': 1,
 'observed': [21],
 'required': [19]}

In [44]:
x = find_next_round_size(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule)

In [45]:
x

{'detailed': {'A-B': {'pstop_goal': [0.5, 0.7, 0.9],
   'next_round_sizes': [32, 48, 69],
   'prob_stop': [0.53551331360561272,
    0.76101184637116392,
    0.90211353611617051]}},
 'future_round_sizes': [32, 48, 69]}

In [46]:
round_schedule = a.round_schedule

In [47]:
round_schedule

[32]

In [48]:
a.add_round_schedule(round_schedule)

In [49]:
a.round_schedule

[32]

In [50]:
x = a.find_next_round_size(pstop_goal)

In [51]:
x

{'detailed': {'A-B': {'pstop_goal': [0.5, 0.7, 0.9],
   'next_round_sizes': [55, 76, 109],
   'prob_stop': [0.77772724415552097,
    0.86072691525663891,
    0.95742964963751509]}},
 'future_round_sizes': [55, 76, 109]}

In [52]:
below_kmin = max(r['required']) - max(r['observed'])

In [53]:
(max(r['required']), max(r['observed']))

(19, 21)

In [54]:
future_round_sizes = x['future_round_sizes']

In [55]:
below_kmin

-2

In [56]:
future_round_sizes

[55, 76, 109]

In [57]:
list(map(lambda x: x - max(round_schedule) + 2 * below_kmin, future_round_sizes))

[19, 40, 73]

# Try to reproduce R2B2/Athena vs BRAVO
Sample Sizes for 90% probability of ending a Ballot Polling Audit of 2016 statewide Presidential contest

with risk limit 0.1, larger margins


In [58]:
# Read in data from 2016
election_2016 = json.load(open('data/2016_election.json'))

In [59]:
election_2016['Alabama']

{'contests': {'presidential': {'winners': 1,
   'candidates': ['Clinton', 'Trump'],
   'results': [729547, 1318255],
   'ballots_cast': 2123372,
   'state_id': 1,
   'margin': -0.2874828718792149}}}

In [60]:
def sample_state(state):
    "Return sample information for given state from 2016"

    election = election_2016[state]
    candidates = election['contests']['presidential']['candidates']
    results = election['contests']['presidential']['results']
    ballots_cast = election['contests']['presidential']['ballots_cast']
    athena_sample = sample90v(results[0], results[1], ballots_cast)
    return athena_sample

In [61]:
states = ['Alabama', 'Maryland', 'New York', 'Rhode Island', 'New Jersey', 'Ohio', 'Virginia',
          'Georgia', 'North Carolina', 'Arizona', 'Nevada' ]
# skip 'Minnesota', 'Florida', 'Wisconsin', 'Pennsylvania', 'Michigan']

In [62]:
athena_results = {}
with redirect_output("debug_output.txt"):
  for state in states:
    athena_results[state] = sample_state(state)

In [63]:
{s: r[0]  for s, r in athena_results.items()}

{'Alabama': 90,
 'Maryland': 94,
 'New York': 132,
 'Rhode Island': 288,
 'New Jersey': 346,
 'Ohio': 1024,
 'Virginia': 2365,
 'Georgia': 2606,
 'North Carolina': 5117,
 'Arizona': 5313,
 'Nevada': 11389}

# Misc snippets of code

In [64]:
x = sample_state('Alabama')

In [65]:
x

(90,
 {'detailed': {'A-B': {'pstop_goal': [0.9],
    'next_round_sizes': [90],
    'prob_stop': [0.90545039854766196]}},
  'future_round_sizes': [90]})

In [66]:
e3 = find_next_round_size(audit_type, alpha, delta, ["A", "B", "C"], [600, 300, 100], ballots_cast, winners, name, model, pstop_goal, round_schedule)

In [67]:
e3

{'detailed': {'A-B': {'pstop_goal': [0.5, 0.7, 0.9],
   'next_round_sizes': [55, 76, 109],
   'prob_stop': [0.77772724415552097,
    0.86072691525663891,
    0.95742964963751509]},
  'A-C': {'pstop_goal': [0.5, 0.7, 0.9],
   'next_round_sizes': [48, 52, 63],
   'prob_stop': [0.99103069660802356,
    0.99388751870684733,
    0.99835662863082941]}},
 'future_round_sizes': [55, 76, 109]}

In [68]:
election = {
    "alpha": alpha,
    "delta": delta,
    "candidates": candidates,
    "results": results,
    "ballots_cast": ballots_cast,
    "winners": winners,
    "name": name,
    "model": model,
    "pstop": pstop_goal,
}

# Ignore the rest - earlier work by hand

In [69]:
below_kmin = max(r['required']) - max(r['observed'])

In [70]:
below_kmin

-2

In [71]:
(max(r['required']), max(r['observed']))

(19, 21)

In [72]:
x = a.find_next_round_size(pstop_goal)

In [73]:
future_round_sizes = x['future_round_sizes']

In [74]:
future_round_sizes

[55, 76, 109]

In [75]:
incremental_round_sizes = list(map(lambda x: x - max(a.round_schedule) + 2 * below_kmin, future_round_sizes))

In [76]:
incremental_round_sizes

[19, 40, 73]

In [77]:
incremental_sample_size = incremental_round_sizes[0]

In [78]:
incremental_sample_size

19

In [79]:
a.add_round_schedule(a.round_schedule + [max(a.round_schedule) + incremental_sample_size])

In [80]:
print_attributes(a)

{'alpha': 0.1,
 'audit_kmins': [19],
 'audit_observations': [],
 'audit_type': 'ATHENA',
 'delta': 1.0,
 'election': <athena.election.Election object at 0x7f678416fda0>,
 'elections': [],
 'round_schedule': [32, 51]}


In [81]:
winner_shares += [a.round_schedule[-1] // 2]

In [82]:
winner_shares

[56, 122, 208, 25]

In [83]:
r = a.find_risk(winner_shares)

IndexError: index 56 is out of bounds for axis 0 with size 33

In [ ]:
r

## Round 3

In [ ]:
below_kmin = max(r['required']) - max(r['observed'])

In [ ]:
below_kmin

In [ ]:
(max(r['required']), max(r['observed']))

In [ ]:
x = a.find_next_round_size(pstop_goal)

In [ ]:
future_round_sizes = x['future_round_sizes']

In [ ]:
future_round_sizes

In [ ]:
incremental_round_sizes = list(map(lambda x: x - max(a.round_schedule) + 2 * below_kmin, future_round_sizes))

In [ ]:
incremental_round_sizes

Tracks interactive output so far: https://gist.github.com/nealmcb/0a165b790b6732096c89535d66aab976

[not true? ] For some reason, need to create a new Audit object with round schedule, can't just update round schedule in existing Audit object (?)

In [ ]:
a = make_audit(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule)

In [ ]:
x = a.find_next_round_size(pstop_goal)

In [ ]:
x

In [ ]:
future_round_sizes = x['future_round_sizes']

In [ ]:
future_round_sizes

doesn't match: 112, 184.  should be 226 260 328?

In [ ]:
future_round_sizes

In [ ]:
list(map(lambda x: x - max(round_schedule) + 2 * below_kmin, future_round_sizes))

In [ ]:
max(round_schedule)

dups...

In [ ]:
round_schedule = a.round_schedule

In [ ]:
a = make_audit(audit_type, alpha, delta, candidates, results, ballots_cast, winners, name, model, pstop_goal, round_schedule)

In [ ]:
print_attributes(a)

In [ ]:
print_attributes(a.election)